## Download the latest Bitcoin data from Bitstamp

In [23]:
import requests
import json
import time
import pandas as pd
import datetime

In [24]:

def dateString(seconds):
    return datetime.datetime.fromtimestamp(seconds).strftime('%Y-%m-%d %H:%M:%S')

In [39]:


# Set the start and end times for the API calls
start_time = 1313798400 # August 20st, 2011 in Unix timestamp format
end_time =   time.time() # Epoch seconds of the current execution time.

# Set the symbol and interval for the API calls
symbol = 'btcusd'
secondsOfAday = 86400
intervalToLoad = secondsOfAday * 7 * 26 # 26 weeks (half a year)

# Create an empty list to store the results
results = []

calls = 0

# Loop through the time range in 1 week intervals and call the API
while start_time < end_time:
    calls += 1
    
    # Build the API endpoint URL
    # see: https://www.bitstamp.net/api/#ohlc_data
    url = f'https://www.bitstamp.net/api/v2/ohlc/{symbol}?step={secondsOfAday}&start={start_time}&end={start_time + intervalToLoad}&limit=1000'

    # Make the API call
    response = requests.get(url)

    # Parse the JSON response and append the results to the list
    results += json.loads(response.text)['data']['ohlc']

    # Increment the start time by intervalToLoad
    start_time += intervalToLoad
    
    print(calls)

    # Pause for 1 second between API calls to avoid rate limiting
    time.sleep(1)
    
# Bitstamp seems to return data that is out of the specified start and end date.
# E.g. with this call: https://www.bitstamp.net/api/v2/ohlc/btcusd?step=86400&start=1313971200&end=1314057600&limit=1000
# the start is 2011-08-22 but the first entry of the result is for the 2011-08-18.
# This results in duplicated entries. To avoid storing that it will be removed with Pandas:
results = pd.json_normalize(results).drop_duplicates(subset=['timestamp'])

# Write the concatenated results to a file
results.to_csv('bitstamp-btc-ohlc.csv', index=False)
#with open('bitstamp-btc-ohlc.json', 'w') as f:
#    json.dump(results, f)
    



1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24


In [41]:
pd.read_csv('bitstamp-btc-ohlc.csv')

,close,high,low,open,timestamp,volume
0,10.90,10.90,10.90,10.90,1313625600,0.489908
1,11.69,11.85,11.15,11.85,1313712000,1.926578
2,11.70,11.70,11.70,11.70,1313798400,0.085470
3,11.70,11.70,11.70,11.70,1313884800,0.085470
4,11.70,11.70,11.70,11.70,1313971200,0.028632
...,...,...,...,...,...,...
4282,27650.00,27841.00,27372.00,27688.00,1683590400,1573.530765
4283,27618.00,28328.00,26842.00,27648.00,1683676800,2848.942523
4284,26991.00,27640.00,26720.00,27624.00,1683763200,2126.871093
4285,26811.00,27093.00,25800.00,26987.00,1683849600,4842.982461


In [36]:
pd.read_json('bitstamp-btc-ohlc.json')

,close,high,low,open,timestamp,volume
0,10.90,10.90,10.90,10.90,2011-08-18,0.489908
1,11.69,11.85,11.15,11.85,2011-08-19,1.926578
2,11.70,11.70,11.70,11.70,2011-08-20,0.085470
3,11.70,11.70,11.70,11.70,2011-08-21,0.085470
4,11.70,11.70,11.70,11.70,2011-08-22,0.028632
...,...,...,...,...,...,...
21645,29333.00,29600.00,28922.00,29486.00,2023-04-28,2449.074977
21646,29247.00,29460.00,29061.00,29339.00,2023-04-29,932.479612
21647,29235.00,29961.00,29115.00,29252.00,2023-04-30,1415.255061
21648,28079.00,29344.00,27666.00,29254.00,2023-05-01,2913.862834


In [48]:

btc_ohlc = pd.read_csv('bitstamp-btc-ohlc.csv')
btc_ohlc = btc_ohlc.loc[:, ['timestamp', 'close']].copy()


In [49]:
btc_ohlc

,timestamp,close
0,1313625600,10.90
1,1313712000,11.69
2,1313798400,11.70
3,1313884800,11.70
4,1313971200,11.70
...,...,...
4282,1683590400,27650.00
4283,1683676800,27618.00
4284,1683763200,26991.00
4285,1683849600,26811.00


In [50]:
pd.to_datetime(btc_ohlc['timestamp'], unit='s').dt.date

0       2011-08-18
1       2011-08-19
2       2011-08-20
3       2011-08-21
4       2011-08-22
           ...    
4282    2023-05-09
4283    2023-05-10
4284    2023-05-11
4285    2023-05-12
4286    2023-05-13
Name: timestamp, Length: 4287, dtype: object

In [ ]:
btc_ohlc['timestamp'] = pd.to_datetime(btc_ohlc['timestamp'], unit='s').dt.date